[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# Covid-19 Vaccination Amounts With Deaths

During the pandemic misinformation surrounding vaccination effectiveness was widespread. The following graph shows the daily deaths within 28 days of a positive Covid 19 test as well as the amount of people vaccinated. Press the "Refresh Data" button to refresh the data, and use the drop down menu to switch between view modes.

In [1]:
# Importing necessary modules
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# loading data from json file
with open("vaccineData.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

In [4]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    # extracting the actual data from the data variable, which is a dictionary of dictionaries. 
    # the actual data is stored in the value associated witht he key 'data' in the highest dictionary
    datalist = rawdata['data']

    # reversing the order of the datalist such that the data entries are in order of increasing date
    datalist.reverse()

    # first extracting the dates and sorting. As the dates are in year first format we can simply
    # use dates.sort() to sort the list.
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()

    # defining a function which converts the dates into pandas format
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    # extracting the first and last date to inform us what time frame we are investigating
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    # creating our data frame, specifying the columns included and the date range of the data.
    # note that the date range may include dates for which we have no data points.
    index=pd.date_range(startdate, enddate, freq='D')
    vaccineDataDF=pd.DataFrame(index=index, columns=['vax1', 'vax2', 'vax3', 'vaxTotal', 'deaths'])

    # populating our dataframe with our data

    # the dictionary defined below allowes us to keep track of all of the vaxination total entries. As these stats
    # are naturally cummulative we wish to replace any None values with the maximum of the values that have come
    # before it. This variable will allow us to do that.
    maxVaxTracker = {'vax1':[0], 'vax2':[0], 'vax3':[0], 'vaxTotal':[0]}
    for entry in datalist: # each entry is a dictionary with date, vax1, vax2, vax3, vaxTotal and deaths stats
        date=parse_date(entry['date']) # converting the date for the data point into the pandas format
        for column in ['vax1', 'vax2', 'vax3', 'vaxTotal', 'deaths']:
            # adding to the tracker dictionary when the column is associated with a vaxination tracker. We add a zero
            # if the value is none, and when we request the max of one of the lists in this dictionary, the zero will
            # not be returned
            if column!='deaths':
                if entry[column]!=None:
                    maxVaxTracker[column].append(entry[column])
                else:
                    maxVaxTracker[column].append(0)
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(vaccineDataDF.loc[date, column]): 
                # replace None with 0 in our data for the deaths column only
                if column == 'deaths':
                    value= float(entry[column]) if entry[column]!=None else 0.0
                # if the column is not the deaths column then we replace the none with the max of the tracker list.
                # Since the data is iterated through with each data point having an increased date than the previous,
                # the max of the tracker list is the largest value before the given None datapoint, i.e the last non
                # None value (as the vaccination stats are all cumulative).
                else:
                    value= float(entry[column]) if entry[column]!=None else max(maxVaxTracker[column])
                # inserting our datapoint into the data frame
                vaccineDataDF.loc[date, column]=value

    # fill in any remaining "holes" due to missing dates
    vaccineDataDF.fillna(0.0, inplace=True)
    return vaccineDataDF

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting

In [5]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    # defining filters such that we get data for the whole of the UK
    filters = [
    'areaType=overview' # This line specifies we are looking for data from the United Kingdom as a whole.
    ]
    
    # selecting the data we wish to extract from the PHE API
    structure = {
    'date':'date', # Date
    'vax1':'cumVaccinationFirstDoseUptakeByPublishDatePercentage', # Cumulative vaccination 1st dose uptake by publish date percentage
    'vax2':'cumVaccinationSecondDoseUptakeByPublishDatePercentage', # Cumulative vaccination 2nd dose uptake by publish date percentage
    'vax3':'cumVaccinationThirdInjectionUptakeByPublishDatePercentage', # Cumulative percentage of people vaccinated with a booster dose plus people vaccinated with third dose by publish date
    'vaxTotal':'cumVaccinesGivenByPublishDate', # Cumulative vaccines given by publish date
    'deaths':'newDeaths28DaysByPublishDate', # New daily deaths within 28 days of a positive test
    }
    
    api = Cov19API(filters=filters, structure=structure) # instanciating the Cov19API class in order to access the API
    vaccineData = api.get_json() # getting the data from the api and storing it in a variable
    return vaccineData # return data read from the API

In [6]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Getting fresh data from the API. If an error occurs, we change the style of the button to reflect that
    # something has gone wrong but we still raise the error.
    try:
        apidata=access_api()
        # wrangling the data and overwriting the dataframe for plotting
        global df
        df=wrangle_data(apidata)
        # the graph won't refresh until the user interacts with the widget.
        # this function simulates the interaction.
        refresh_graph()
        # in the case of a successful refresh and API download, the button is changed in appearance and is disabled.
        apibutton.icon="check"
        apibutton.button_style="success" # turns button green
        apibutton.description="Data Refreshed"
        apibutton.disabled=True # disables button so excesive button spamming is not problematic
        apibutton.tooltip="Data has been refreshed" # changing hover over tooltip of button
        # apibutton.disabled=True
        
    except Exception as e:
        # In the case of a generic error the buttons appearance is changed, but the button is not disabled. We also
        # still raise the exception to maintain good practice and allow building on this code.
        apibutton.button_style="danger" #changes button red in the case of an error
        apibutton.icon="warning" # displays error icon on button
        apibutton.description="Error"
        apibutton.tooltip="An error occurred"
        raise(e)


# Creating the refresh button and assigning appropriate attributes.  
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# registering the button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

#displaying the button
display(apibutton)

# run all cells before clicking on this button

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

In [7]:
# defining the function that will plot the data showing how deaths changed as more people became vaccinated.
def vaccine_graph(graphcolumns):
    # storing the vaxMode objects value attributes in variable current for ease of use
    current = vaxMode.value
    # creating a figure and 2 sets of y axes which use subplots to allow multiple axis plotting
    fig,ax = plt.subplots()
    ax2 = ax.twinx()
    
    if current == vaxMode.options[1]: # in the case of selecting to display total vaccinations and deaths
        # plotting deaths on the first y axis
        ax.plot(df['deaths'], color="blue")
        # labelling axes
        ax.set_xlabel("Date", fontsize = 14)
        # plotting total vaccines given on the other y axis
        ax2.plot(df['vaxTotal'], color="orange")
        # labelling other y axis
        ax2.set_ylabel("Total number of vaccinations",
              color="orange",
              fontsize=14)
    elif current == vaxMode.options[0]: # in the case of selecting to display individual rates we plot deaths
        # on one y axis and 3 on the second y axis
        
        # plotting deaths on left y axis
        ax.plot(df['deaths'], color="blue")
        # setting labels
        ax.set_xlabel("Date", fontsize = 14)
        # plotting all 3 individual vax stats on the right hand y axis
        ax2.plot(df['vax1'], color="red", label="1st Dose")
        ax2.plot(df['vax2'], color="orange", label= "2nd Dose")
        ax2.plot(df['vax3'], color="yellow", label="3rd Dose")
        # setting label
        ax2.set_ylabel("Vaccination percentage by dose",
              color="orange",
              fontsize=14)
        #enabling a legend to distinguish between the 3 different vaccination wave stats
        ax2.legend()
    ax.set_ylabel("Number of deaths on given day",
      color="blue",
      fontsize=14)
    # itterating through tick/mark labels and rotating each one by 45 degrees in order to improve readability
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    plt.show()

# creating the vaxMode widget dropdown menu and specifying the attributes.
vaxMode=wdg.Dropdown(
    options=['Individual rates of vaccination', 'Total number of vaccinations'], # options available
    value='Individual rates of vaccination', # initial value
    rows=2, # rows of the selection box
    description='View Mode', # label
    disabled=False
)

# this function refreshes the graph by drawing the other mode and then redraws the previous mode.
def refresh_graph():
    """ Changing the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. """
    current=vaxMode.value
    if current==vaxMode.options[0]:
        other=vaxMode.options[1]
    else:
        other=vaxMode.options[0]
    vaxMode.value=other # forces the redraw
    vaxMode.value=current # now we can change it back
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(vaccine_graph, {'graphcolumns': vaxMode})

# displays the graph
display(vaxMode, output)

Dropdown(description='View Mode', options=('Individual rates of vaccination', 'Total number of vaccinations'),…

Output()

**Author and Copyright Notice** Created by Ben Malpas using work by Fabrizio Smeraldi. 
*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*